In [ ]:
%%html
<style>
  .container {
      width: 100%;
  }
</style>

# Statistiken

Da die KI-Versionen im Folgenden oft referenziert werden, wird hier noch einmal eine Übersicht über die verschiedenen Implementierungen gegeben.

- `AI-Base-Class`: Die abstrakte Basisklasse für alle AI-Versionen, welche das Eröffnungs- und Endspiel implementiert.
- `Exercise01AI`: Die erste Version der AI, welche im Mittelspiel zufällige Züge auswählt.
- `Exercise02AI`: Die zweite Version der AI, welche im Mittelspiel Züge mithilfe des Minimax-Algorithmus und dem einfachen Materialwert auswählt.
- `Exercise03AI`: Die dritte Version der AI, welche im Mittelspiel Züge mithilfe des Minimax-Algorithmus und der `Simplified Evaluation Function` auswählt.
- `Exercise04AI`: Die vierte Version der AI, welche im Vergleich zur dritten Version mithilfe von Memoisierung beschleunigt wurde.
- `Exercise05AI`: Die fünfte Version der AI, welche die dritte Version um das Alpha-Beta-Pruning erweitert.
- `Exercise06AI`: Die sechste Version der AI, welche die fünfte Version erneut um Memoisierung erweitert.
- `Exercise07AI`: Die siebte Version der AI, welche die sechste Version um Progressive Deepening erweitert.
- `Exercise08AI`: Die achte Version der AI, welche die siebte Version um die Singular Value Extension erweitert.
- `StockfishPlayer`: Ein Spieler, welcher durch die Stockfish Engine gesteuert wird.

## Ressourcenverbrauch der verschiedenen KI-Versionen

Die CPU-Auslastung ist bei jeder KI-Version maximal, das heißt es wird durchgängig die verfügbare Rechenleistung genutzt. Die KI ist aktuell nicht parallelisiert, das heißt es wird jeweils nur ein CPU-Kern genutzt. Der Festplattenspeicher bewegt sich innerhalb von wenigen MB Verbrauch und ist somit vernachlässigbar. Die kritische Ressource ist der Arbeitsspeicher. Der genutzte Speicher unterscheidet sich je nach KI-Version und Konfiguration stark und ist (neben der Berechnungszeit) ein limitierender Faktor.

### Vergleich des RAM-Verbrauchs

Im Folgenden wird der Verbrauch des Arbeitsspeichers für die KI-Version 5 bis 8 verglichen. Die Versionen 1 bis 4 verfügen noch nicht über das Alpha-Beta-Pruning welches die Berechnungen stark beschleuningt, daher sind sie für einen Vergleich weniger interessant.

#### Messaufbau

Für jede gewählte Konfiguration und KI-Version wurden drei Testspiele gegen die Stockfish KI gespielt. Die Spiele wurden ohne GUI auf auf einem Debian Linux gestartet. Der virtuelle Server wurde exklusiv für diesen Test erstellt und verfügt über 32 GB Arbeitsspeicher.  
Während den Spielen wurde der freie Arbeitsspeicher des Gesamtsystems im Abstand von 30 Sekunden gemessen. Der Verbrauch des Systems im Leerlauf wurde anschließend herausgerechnet. Die Messungenauigkeit wird auf etwa 15 MB geschätzt (maximale Abweichungen im Leerlauf).

#### Erläuterungen zur Tabelle

Für den RAM wurde jeweils das Maximum über alle drei Spiele ermittelt. Hierbei ist anzumerken, dass die Maximalwerte für ein einzelnes Spiel teilweise um mehr als 50 % vom Mittelwert der Maximalwerte aller Spiele abweicht, die Varianz ist hier also sehr hoch. Die `Exercise05AI` besitzt keine Memoisierung, somit gibt es keine Messung für den Cache. Bei der `Exercise08AI` wurd die maximale Tiefe für die Singular Value Extension jeweils um 1 höher gewählt als die normale Tiefe (minimaler Wert) um so weiterhin vergleichbar zu sein.

| Tiefe | Exercise05AI RAM | Exercise06AI RAM | Exercise07AI RAM | Exercise08AI RAM |
|-------|------------------|------------------|------------------|------------------|
| 3     | 128 MB           | 128 MB           | 305 MB           | 635 MB           |
| 4     | 138 MB           | 305 MB           | 658 MB           | 1010 MB          |
| 5     | 186 MB           | 1189 MB          | 3240 MB          | 2973 MB          |

Es zeigt sich, dass die Memoisierung den größten Einfluss auf den Arbeitsspeicher hat. Während der Verbrauch ohne Memoisierung nur leicht ansteigt (`Exercise05AI`), passiert dies deutlich stärker, wenn Memoisierung eingesetzt wird (`Exercise06AI`). Bei der iterativen Suche (`Exercise07AI`) erhöht sich der Verbrauch abermals und bleibt bei der `Exercise08AI` in etwa gleich.

#### Exemplarischer Verlauf der Arbeitsspeicherbelastung

Das folgende Bild zeigt exemplarisch den Verlauf der RAM-Nutzung für ein Spiel der `Exercise08AI` gegen Stockfish. Gemessen wurde jeweils im Abstand von 30 Sekunden bei einer Suchtiefe von 4 und maximaler SVE-Tiefe von 8. Die SVE-Tiefe ist die Tiefe bis zu der die Singular Value Extension durchgeführt wird. Ist diese Tiefe erreicht, wird die Suche in diesem Teilbaum abgebrochen. Die vertikalen Striche im Diagramm markieren das Ende eines Suchvorganges, das heißt einen abgeschlossenen Halbzug. Da Stockfish jeweils nur 0,1 Sekunden für den Zug benötigt kann der zeitliche Anteil in der Messung vernachlässigt werden. Bei den steilen Anstiegen im Diagramm handelt es sich jeweils um eine Folge von Zügen mit vergleichsweise hoher durchschnittlicher Tiefe, das heißt es wurden viele Stellungen untersucht. Wenn im Diagramm die Auslastung auf das Basisniveau von etwa 200 MB fällt, wurde der Cache nach einem irreversiblen Zug geleert.

<img src="images/RAM_Ex08AI_vs_Stockfish.png" width="1200px" />

#### Arbeitsspeicherbelastung bei höherer Maximaltiefe

Die folgende Tabelle zeigt die Arbeitsspeicherbelastung für die `Exercise08AI` in verschiedenen Konfigurationen. Als Vergleichswert wurde noch die durchschnittliche Zeit für einen Halbzug und die durchschnittliche Suchtiefe hinzugefügt. Die durchschnittliche Tiefe wird als Summe der Suchtiefe bei jedem Blatt des Suchbaumes, geteilt durch die Anzahl der Summanden definiert. Ein Blatt entsteht, sobald eine der Abbruchbedingungen für die Rekursion zutrifft. Es zeigt sich, dass die RAM-Auslastung bei höherer Maximaltiefe stark ansteigt und bei einer SVE-Tiefe von 12 die Suche auf den meisten aktuellen Rechnern bereits zu viel Arbeitsspeicher benötigt. Auch die Zeit pro Zug steigt stark. Hierbei ist zu beachten, dass die Zugzeit vor allem bei den ersten Zügen sehr hoch ist und bei einer SVE-Tiefe von 12 beispielsweise bis zu 52 Minuten für einzelne Züge möglich sind.

| Max. SVE Tiefe | Tiefe | RAM        | Durchschnittliche Berechnungszeit pro Zug | Durchschnittlich erreichte Suchtiefe |
|----------------|-------|------------|-------------------------------------------|--------------------------------------|
| 4              | 3     | 635 MB     | 0,75 s                                    | 3,49                                 |
| 5              | 4     | 1010 MB    | 10,29 s                                   | 4,27                                 |
| 6              | 5     | 2431 MB    | 51,44 s                                   | 5,10                                 |
| 8              | 4     | 1756 MB    | 20,96 s                                   | 5,63                                 |
| 10             | 4     | 6695 MB    | 52,38 s                                   | 6,14                                 |
| 12             | 4     | > 31606 MB | 82,18 s                                   | 4,92                                 |

Die Daten zeigen insgesamt einerseits, dass die Memoisierung viel Arbeitsspeicher erfordert, aber dieser auch effektiv wieder freigegeben wird, sobald der Cache durch einen irreversiblen Zug nutzlos wird. Die zweite Tabelle zeigt aber ebenso, dass der Arbeitsspeicher durchaus ein Limit für die Suche darstellen kann (wenn auch nur bei einer sehr langen Rechenzeit).

## Entwicklung der einzelnen KIs


Dieses Kapitel stellt die Entwicklung der einzelnen KI-Versionen dar, wobei die erste KI-Iteration übergangen wird, da diese nur zufällige Züge auswählt und somit für keine andere KI ein ernsthafter Gegner ist. Im Folgenden ist eine Zusammenfassung von Spielen des Stockfish Spielers gegen alle implementierten KI-Iterationen zu sehen.

<img src="images/StockfishResultsDepth4.png" width="800px" />

Aus dieser Statistik ist zu entnehmen, dass die `Exercise02AI` das schlechteste Resultat liefert, da deren Bewertung eines Zugs nur über den einfachen Materialwert stattfindet. Alle folgenden KI-Iterationen sollten sich nicht in ihrer Spielstärke unterscheiden, sondern lediglich in ihrer Performanz. Um einen fairen Vergleich sicherzustellen, wird der `max_depth` Parameter der `Exercise08AI`, also die maximale Suchtiefe der Singular Value Extension (SVE), immer der Suchtiefe der anderen KI-Versionen angepasst. Die SVE wird somit nicht effektiv in dieser Diskussion verwendet. Mit Ausnahme der Ergebnisse der `Exercise07AI` wird diese Annahme auch von den Statistiken unterstützt. Diese Ausnahme ist vermutlich auf Messungenauigkeiten zurückzuführen, welche durch die Sortierung der Züge durch das Progressive Deepening auftreten können.

Im folgenden Diagramm wird die maximale Suchtiefe der SVE auf acht gesetzt, die generelle Suchtiefe beträgt weiterhin vier.

<img src="images/Ex08ResultsDepth4MaxDepth8.png" width="800px" />

In diesem Diagramm spiegelt das Resultat die erwarteten Werte wider. Die `Exercise08AI` gewinnt aufgrund der SVE gegen ihre Vorgängerversionen oder läuft aufgrund der Zugwiederholung in ein Remis.


### Zugzeit der KIs

Bei den einzelnen Entwicklungsstufen der KI lassen sich klare zeitliche Unterschiede erkennen. Während die Memoisierung einen gewissen, aber leider nur geringen Geschwindigkeitsvorteil ermöglicht, bietet das Alpha-Beta-Pruning die größte Zeitersparnis.

Als `max_depth` Parameter (maximale SVE-Tiefe) wurde bei den folgenden Statistiken für die finale KI-Version die jeweilige Suchtiefe verwendet.
Wie in den nachfolgenden Abbildungen zu sehen ist, erhöhen sowohl die Einführung des Progressive Deepenings als auch der Singular Value Extension (SVE) den benötigten Zeitaufwand immens. Im Falle des Progressive Deepening wird der Zeitaufwand bei einer Tiefe von 4 um über 100 % erhöht. Auch wenn der Vorteil der SVE durch die gegebene `max_depth` neutralisiert wird, beansprucht der zusätzliche Code dennoch zusätzliche Leistung. Bei einer Suchtiefe von 3 ist die finale KI-Version über 50 % langsamer, als die `Exercise07AI`. Durch die besser Zugsortierung ändert sich dieser Punkt allerdings mit der Suchtiefe 4, bei der die `Exercise08AI` ihre Vorgängerversion zeitlich überholt.

Das Progressive Deepening sorgt für eine andere Sortierung der Züge, weshalb andere Ergebnisse möglich sind, aber das eigentliche Ziel einer Reduzierung der Berechnungsdauer wurde nicht erreicht.

Bei einer Depth von drei ist die durchschnittliche Zugzeit aller KIs sehr gering. Während die `Exercise03AI` mit 1,17 Sekunden die längste Rechendauer vorweist, ist die `Exercise05AI` mit durchschnittlich 0,24 Sekunden die schnellste KI.

<img src="images/Depth3AvgDuration.png" width="900px" />

Die Berechnungsdauer wächst exponentiell mit steigender Suchtiefe. Bei einer Tiefe von 4 ist die sechste KI Version mit 1,97 Sekunden die schnellste KI. Mit durchschnittlich 27,74 Sekunden pro Zug ist `Exercise02AI` die KI mit der längsten Berechnungsdauer. Da die dritte KI-Generation nur minimal schneller ist und auch die `Exercise04AI` durchschnittlich über zwölf Sekunden rechnet, werden diese beiden Versionen nicht im folgenden Diagramm angezeigt, um die Darstellung überschaubarer zu gestalten. 

<img src="images/Depth4AvgDuration.png" width="900px" />

#### Exercise08AI

Durch die Verwendung der Singular Value Extension und dem dazugehörigen Parameter `max_depth` wird die finale Version der KI separat betrachtet und mit unterschiedlichen Werten für `max_depth` parametrisiert. In dem oben dargestellten Diagramm, welches die Spielergebnisse bei einer `max_depth` von acht anzeigt, beträgt die durchschnittliche Rechendauer der `Exercise08AI` pro Zug 8,48 Sekunden.

### Anzahl der berechneten Knoten

Um den tatsächlichen Fortschritt der einzelnen KI-Iterationen darzustellen, wird im Folgenden die Anzahl der berechneten Knoten für die Suchtiefen drei und vier untersucht, da dieser Wert im Gegensatz zu der durchschnittlichen Berechnungsdauer deterministisch ist.

Während sowohl die zweite als auch die dritte KI-Version über 25.000 Stellungen untersuchen, wird durch die in der `Exercise04AI` implementierte Memoisierung die Anzahl der untersuchten Stellungen auf unter 13.914 reduziert, was fast einer Halbierung entspricht.
Den größten Vorteil erzielt das in `Exercise05AI` implementierte Alpha-Beta-Pruning, welches die Anzahl der berechneten Knoten auf 4418 reduziert. Die `Exercise06AI` hat mit 3143 Knoten den quantitativ geringsten Rechenaufwand, da ihre Nachfolgerversion durch das Progressive Deepening und die SVE wieder eine größere Anzahl an Stellungen berechnen.
<img src="images/Depth3AvgNodes.png" width="900px" />

Auch mit einer erhöhten Suchtiefe von vier verändert sich dieses Bild nicht. Durch das exponentielle Wachstum der Anzahl der berechneten Knoten werden die Vorgängerversion der fünften KI-Iterationen aus dem nachfolgenden Diagramm entfernt, die dritte KI-Generation hat hierbei mit 882.048 Stellungen den quantitativ höchsten Rechenaufwand.
<img src="images/Depth4AvgNodes.png" width="900px" />

Bei Betrachtung der finalen KI Version mit einer Suchtiefe von vier und einer maximalen SVE Tiefe von acht berechnet diese durchschnittlich 173.416 Stellungen, womit sie noch immer einen quantitativ niedrigeren Rechenaufwand als die `Exercise04AI` mit 192.333 Knoten hat.

## Vergleich von Sieg und Niederlage

Für den Vergleich von Sieg und Niederlage wurden jeweils mindestens 10 Spiele gespielt. Die Diagramme zeigen die Gewinne der einzelnen KI-Versionen gegen Stockfish in Prozent. Bei der Generierung der Statistiken wurde Stockfish auf eine ELO-Zahl von `1500` und eine Zug-Zeit von `0.1` Sekunden limitiert. Die ELO-Zahl von Stockfish wurde von den Entwicklern mit einer Bedenkzeit von `60s+0.6s` und in der `CCRL 40/4` kalibriert. Aufgrund der Menge an benötigten Spielen wurde nur mit den Tiefen `3` und `4` gespielt. Die ersten zwei Diagramme zeigen die Ergebnisse mit Tiefe 3, die letzten zwei mit einer Tiefe von 4.

Alle KI-Versionen gewinnen mit einer Tiefe von 4 ungefähr doppelt so oft als mit einer Tiefe von 3. Schnell erkennbar ist in allen vier der folgenden Diagrammen, dass `Exercise01AI` durchweg verliert. Dies liegt an der Auswahl zufälliger Züge im Mittelspiel. Die `Exercise02AI`, welche im Mittelspiel Züge mithilfe des Minimax-Algorithmus und dem einfachen Materialwert auswählt, gewinnt mit einer Tiefe von 3 durchschnittlich 4 % der Spiele, mit einer Tiefe von 4 schon durchschnittlich 16 %. Hierbei hängt das Ergebnis stark von der Farbe ab, mit der die KI spielt. `Exercise03AI`, welche im Mittelspiel Züge mithilfe des Minimax-Algorithmus und der `Simplified Evaluation Function` auswählt, wird vor allem weiß spielend ungefähr doppelt so gut. Die Beschleunigung durch die Memoisierung kommt bei `Exercise04AI` zu tragen, diese gewinnt auf einer Tiefe von 4 im Schnitt bereits die Hälfte der Spiele.

Die KI-Versionen `Exercise05AI`, `Exercise06AI` und `Exercise07AI` spielen verhältnismäßig sehr ähnlich. Dies ist darauf zurückzuführen, dass hierbei nicht das "Können", sondern durch das implementierte Alpha-Beta-Pruning, die erneute Memoisierung und das Progressive Deepening lediglich die Geschwindigkeit optimiert wurde. 

Mit mehr als 54 % gewonnener Spiele auf einer Tiefe von 4 besiegt `Exercise08AI` Stockfish. Diese Verbesserung kann auf die implementierte Singular Value Extension zurückgeführt werden.

<img src="images/ResultsWhiteDepth3.png" width="850px" />
<img src="images/ResultsBlackDepth3.png" width="850px" />
<img src="images/ResultsWhiteDepth4.png" width="850px" />
<img src="images/ResultsBlackDepth4.png" width="850px" />

## Die Effektivität des Cache

Während jedes Spiels wurde pro Halbzug der Zugriff auf den implementierten Cache gemessen. Hierfür wurden die Variablen `cache_hits` und `cache_tries` für jede Farbe implementiert. Nach einem abgeschlossenen Spiel konnten so über alle `cache_hits` und `cache_tries` der jeweiligen Farbe die Summe gebildet werden und folglich mithilfe der Formel

$$ \textrm{effectiveness} = \frac{\textrm{cache_hits}}{\textrm{cache_tries}} $$

für ein gesamtes Spiel die Effektivität des Caches in Prozent je Farbe berechnet werden. Die zwei Diagramme unten zeigen den Mittelwert der Effektivität der jeweiligen KI-Versionen bei mehr als 50 Spielen der Tiefen 3 und 4.

<img src="images/CacheHitsWhite.png" width="850px" />
<img src="images/CacheHitsBlack.png" width="850px" />

Interessanterweise hat das Alpha-Beta-Pruning (ab `Exercise06AI`) also die Cache-Effektivität deutlich verringert, während sie in den darauf folgenden Versionen nur noch leicht gesunken ist.